# Map examples

This notebook loads some simple GeoJSON data containing lines for submarine cables from [submarinecablemap.com](http://www.submarinecablemap.com) and displays them on various maps, where you can pan and zoom and, sometimes, also tilt and rotate the map.

## Load dataset

In [ ]:
from IPython.display import JSON
import requests

url = (
    "https://raw.githubusercontent.com/telegeography/www.submarinecablemap.com"
    "/master/web/public/api/v3/cable/cable-geo.json"
)
data = requests.get(url).json()
style_callback = lambda feat: {"color": feat["properties"]["color"]}
JSON(data)

The `data` and `style_callback` identifiers are reused in the cells below.

## Ipyleaflet

In [ ]:
from ipyleaflet import Map, GeoJSON

m = Map(center=[0, 0], zoom=2)
m += GeoJSON(data=data, style_callback=style_callback)
m

## Folium

In [ ]:
import folium

m = folium.Map(location=[0, 0], zoom_start=2)
geojson = folium.GeoJson(data=data, style_function=style_callback)
geojson.add_to(m)
m

## Geemap

This needs a Google Earth Engine registration. The `style_callback` parameter works as expected in geemap>0.9.2.

In [ ]:
import geemap

m = geemap.Map(center=[0, 0], zoom=2, height="450px")
m.add_geojson(data, style_callback=style_callback)
m

## Leafmap

The `style_callback` parameter works as expected in leafmap>0.5.0.

In [ ]:
import leafmap

m = leafmap.Map(center=[0, 0], zoom=2, height="450px")
m.add_geojson(data, style_callback=style_callback)
m

## HERE Map Widget

This needs a registration for a HERE developer account. The code reads the according API key from the ``HEREMAPS_API_KEY`` environment variable. Set it below or provide the API key directly in the code, instead:

In [ ]:
import os
os.environ["HEREMAPS_API_KEY"] = "******"

In [ ]:
import os
from here_map_widget import Map, GeoJSON

style_callback1 = lambda feat: {"strokeColor": feat["properties"]["color"]}
api_key = os.getenv("HEREMAPS_API_KEY")
m = Map(center=[0, 0], zoom=1, api_key=api_key)
m += GeoJSON(data=data, style_callback=style_callback1)
m

## HERE Inspector (ipyleaflet)

In [ ]:
from here.inspector import inspect

inspect(data, zoom=2) # , style=lambda feat: {"color": feat["properties"]["color"]})

## Deck.gl

In [ ]:
import pydeck

def hex_to_rgb(value):
    value = value.lstrip('#')
    return list(int(value[i:i+2], 16) for i in (0, 2, 4))

for feat in data["features"]:
    feat["properties"]["rgbcolor"] = hex_to_rgb(feat["properties"]["color"])

pydeck.settings.custom_libraries = [
    {
        "libraryName": "LabeledGeoJsonLayerLibrary",
        "resourceUri": "https://unpkg.com/pydeck-custom-layer-demo/dist/bundle.js",
    }
]

geojson_layer = pydeck.Layer(
    "LabeledGeoJsonLayer",
    data=data,
    filled=False,
    billboard=False,
    get_line_color="properties.rgbcolor",
    label_size_units=pydeck.types.String("meters"),
    line_width_min_pixels=2,
)

initial_view_state = pydeck.ViewState(latitude=0, longitude=0, zoom=1, max_zoom=16, pitch=0, bearing=0)
r = pydeck.Deck(
    layers=[geojson_layer],
    initial_view_state=initial_view_state,
    map_provider="carto",
    map_style="light"
)
r.to_html("cables_deckgl.html", css_background_color="#333")

## Kepler.gl

In [ ]:
from keplergl import KeplerGl

config = {"version": "v1", "config": {"mapState": {"latitude": 0.0, "longitude": 0.0, "zoom": 1}}}
m = KeplerGl(config=config)
m.add_data(data=data, name="Cables")
m